In [ ]:
!pip install llama-index-core
!pip install llama-index-llms-gemini
!pip install llama-index-embeddings-huggingface
!pip install llama_index.readers.file

In [ ]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os

In [3]:
token_counter = TokenCountingHandler(
    verbose=True
)

Settings.callback_manager = CallbackManager([token_counter])


In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
Settings.chunk_size = 512
Settings.chunk_overlap = 50

documents = SimpleDirectoryReader("./Summary").load_data()
index = VectorStoreIndex.from_documents(documents)

In [6]:
os.environ["GOOGLE_API_KEY"] = ('your_google_api_key')
Settings.llm = Gemini(
    model_name="models/gemini-1.5-pro-latest",
    temperature=0.2
)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=1000)
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="Health__Summary",
            description=(
                "Has information about the Health Summary documents. "
            ),
        ),
    ),
]

agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    max_iterations=100
)

response = agent.chat("list the summary of the documents")
print(str(response))